In [54]:
# Cp x Cq ==> Cq -> Aut(Cp) ==> gcd(|Cq|, |Aut(Cp)|)

In [107]:
# C43 x (C29 x (C7 x (C3 x C2)))
print(gcd(2, euler_phi(3)))
print(gcd(2*3, euler_phi(7)))
print(gcd(2*3*7, euler_phi(29)))
print(gcd(2*3*7*29, euler_phi(43)))
2*3*7*29*43

2
6
14
42


52374

In [76]:
# (Cp x Cq) x Cr 
# Cq -> Aut(Cp)
# Cr -> Aut(Cp x Cq)

75516

In [78]:
# C2 -> Aut(C3)
# 1: 1 -> 2

37758

In [109]:
class Semi:
    
    def __init__(self,a,b,twist):
        self.a = a
        self.b = b
        self.twist = twist
        
    def gen_element(self, x,y):
        return SemiElement(self, x%self.a, y%self.b)
    
class SemiElement:
    
    def __init__(self, semi, x, y):
        self.semi = semi
        self.x = x
        self.y = y
        
    def __neg__(self):
        ...

1218

In [110]:
# Order 252: https://people.maths.bris.ac.uk/~matyd/GroupNames/241/C3%5E2sDic7.html
#   C7⋊(C32⋊C4)
#   G / C7 ~ C32⋊C4
#   G / C7 / C32 ~ C4
#     C4   C32
#   1 < C4  < C32⋊C4 < G
#   G = < a,b,c,d | a3=b3=c14=1, d2=c7, ab=ba, cac-1=a-1, dad-1=ab-1, cbc-1=b-1, dbd-1=a-1b-1, dcd-1=c-1 >
# Order 270: https://people.maths.bris.ac.uk/~matyd/GroupNames/257/He3sD5.html

1218

In [111]:
G=gap.SmallGroup(252,32);

In [112]:
# RCON for C4
RCON = [0, 0, 0, 0, 0, 2, 2, 3, 2, 2, 0, 1, 3, 0, 1, 0, 2, 1, 2, 3, 3, 2, 2, 3, 1, 0, 3, 1, 3, 1, 2, 0]
N_ROUNDS = 10
N_BYTES = 16

def shift_rows(s):
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]
    
def add_round_key(s, k):
    for i in range(4):
        for j in range(4):
            s[i][j] += k[i][j]
            
def mix_single_column(a):
    b1,b2,b3,b4 = (
        2*a[0] + 3*a[1] + 1*a[2] + 1*a[3],
        1*a[0] + 2*a[1] + 3*a[2] + 1*a[3],
        1*a[0] + 1*a[1] + 2*a[2] + 3*a[3],
        3*a[0] + 1*a[1] + 1*a[2] + 2*a[3]
    )
    a[0],a[1],a[2],a[3] = b1,b2,b3,b4
    
def mix_columns(s):
    for i in range(4):
        mix_single_column(s[i])
        
def xor_bytes(a, b):
    return [i+j for i, j in zip(a, b)]

def bytes2matrix(text):
    return [text[i:i+4] for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    return sum(matrix, [])
        
def expand_key(master_key):
    
    key_columns = bytes2matrix(master_key)
    iteration_size = len(master_key) // 4

    i = 1
    while len(key_columns) < (N_ROUNDS + 1) * 4:
        # Copy previous word.
        word = list(key_columns[-1])

        # Perform schedule_core once every "row".
        if len(key_columns) % iteration_size == 0:
            # Circular shift.
            word.append(word.pop(0))
            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            word[0] += RCON[i]
            i += 1

        # XOR with equivalent word from previous iteration.
        word = xor_bytes(word, key_columns[-iteration_size])
        key_columns.append(word)

    # Group key words in 4x4 byte matrices.
    return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

def encrypt_block(key, plaintext):

    assert len(plaintext) == N_BYTES

    plain_state = bytes2matrix(plaintext)
    round_keys = expand_key(key)
    
    add_round_key(plain_state, round_keys[0])

    for i in range(1, N_ROUNDS):
        shift_rows(plain_state)
        mix_columns(plain_state)
        add_round_key(plain_state, round_keys[i])

    shift_rows(plain_state)
    add_round_key(plain_state, round_keys[-1])

    return matrix2bytes(plain_state)

In [113]:
F = Zmod(4)
key = PolynomialRing(F, ["k%d"%d for d in range(16)]).gens()

xk1 = [0, 1, 3, 2, 3, 2, 1, 3, 3, 2, 0, 1, 2, 2, 2, 2]
xp1 = [2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0]
xc1 = [1, 0, 1, 3, 2, 0, 2, 3, 3, 0, 1, 1, 1, 3, 3, 1]

In [114]:
ct = encrypt_block(key,xp1)
mat = matrix(F, [
    [r[k] for k in key] for r in ct
])
c1mod = vector(F, [c-ct[i].constant_coefficient() for i,c in enumerate(xc1)])

k1_c4 = mat.solve_right(c1mod)
assert all(int(x)==y for x,y in zip(k1_c4, xk1))

In [106]:
for i in range(16):
    globals()["k%d"%i] = xk1[i]

In [107]:
print([eval(str(c))%4 for c in ct])
print(xc1)

[1, 0, 1, 3, 2, 0, 2, 3, 3, 0, 1, 1, 1, 3, 3, 1]
[1, 0, 1, 3, 2, 0, 2, 3, 3, 0, 1, 1, 1, 3, 3, 1]
